In [2]:
import json

In [20]:
test = [{"time":i, "value":i*2} for i in range(10)]
# tmp = json.dumps(test) #convert to json

labels = [i for i in range(10)]
temp_values = [i*2 for i in range(10)]
hum_values = [10-i*0.25 for i in range(10)]
water_values = [80-i*0.5 for i in range(10)]

test = {"labels":labels,"temp_values":temp_values,"hum_values":hum_values,"water_values":water_values}

In [21]:
print(test)
# print(tmp)
with open('data.json', 'w') as fp:
    json.dump(test, fp)

{'labels': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'temp_values': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18], 'hum_values': [10.0, 9.75, 9.5, 9.25, 9.0, 8.75, 8.5, 8.25, 8.0, 7.75], 'water_values': [80.0, 79.5, 79.0, 78.5, 78.0, 77.5, 77.0, 76.5, 76.0, 75.5]}
